# Model

> Central diffusion model specification

This is adapted from my [FastAI notes]()

In [23]:
# |default_exp model

In [24]:
# |export
import math
from typing import Sequence, TypeAlias

import torch
from beartype import beartype
from jaxtyping import Float, jaxtyped
from torch import Tensor, nn

In [25]:
# |exports
InFeatureMapTensor: TypeAlias = Float[Tensor, "bs c_in h_in w_in"]
OutFeatureMapTensor: TypeAlias = Float[Tensor, "bs c_out h_out w_out"]
TimeStepTensor: TypeAlias = Float[Tensor, "bs"]  # from 0 to 1
TimeStepEmbeddingTensor: TypeAlias = Float[Tensor, "bs t"]

## Modules

Now, let's get to the module definitions.

The ConvBlock is laid out in the "Preactivation" configuration, like so:

![](https://www.researchgate.net/publication/337691625/figure/fig2/AS:831842160746496@1575338034408/Architecture-of-normal-residual-block-a-and-pre-activation-residual-block-b.jpg)

This does mean that we need to take care in the first block to use a "raw" nn.Conv, because otherwise the activation would discard pixel information.

In [26]:
# |exports
class ConvBlock(nn.Module):
    """Wrapper for a Conv block with normalization and activation"""

    @beartype
    def __init__(
        self,
        c_in: int,
        c_out: int,
        ks: int = 3,
        stride: int = 1,
        act: type[nn.Module] = nn.ReLU,
    ):
        super().__init__()
        self.ks = ks
        self.norm = nn.BatchNorm2d(c_in)
        self.act = act()
        self.conv = nn.Conv2d(
            c_in,
            c_out,
            stride=stride,
            kernel_size=ks,
            padding=ks // 2,
            bias=False,
        )

    @jaxtyped(typechecker=beartype)
    def forward(self, x: InFeatureMapTensor) -> OutFeatureMapTensor:
        x = self.norm(x)
        x = self.act(x)
        x = self.conv(x)
        return x

In [27]:
# |exports
@jaxtyped(typechecker=beartype)
def timestep_embedding(
    ts: TimeStepTensor, emb_dim: int, max_period: int = 10_000
) -> TimeStepEmbeddingTensor:
    exponent = -math.log(max_period) * torch.linspace(
        0, 1, emb_dim // 2, device=ts.device
    )
    embedding = ts[:, None].float() * exponent.exp()[None, :]
    embedding = torch.cat([embedding.sin(), embedding.cos()], dim=-1)
    return embedding

In [28]:
# |exports
class TimeEmbeddingMixer(nn.Module):
    """Incorporate the time embedding into the ResBlock logits"""

    @beartype
    def __init__(self, c_time: int, c_out: int, act: type[nn.Module] = nn.ReLU):
        super().__init__()
        self.lin = nn.Linear(c_time, c_out * 2)
        self.act = act()

    @jaxtyped(typechecker=beartype)
    def forward(
        self, x: InFeatureMapTensor, t_emb: TimeStepEmbeddingTensor
    ) -> OutFeatureMapTensor:
        t_emb = self.lin(self.act(t_emb))[:, :, None, None]
        scale, shift = torch.chunk(t_emb, 2, dim=1)
        return x * (1 + scale) + shift

In [29]:
# |exports
class ResBlock(nn.Module):
    """Conv resblock with the preactivation configuration and time embedding modulation"""

    @beartype
    def __init__(
        self,
        c_time: int,
        c_in: int,
        c_out: int,
        ks: int = 3,
        stride: int = 2,
        act: type[nn.Module] = nn.ReLU,
    ):
        super().__init__()
        self.c_time = c_time
        self.c_in = c_in
        self.c_out = c_out

        self.time_mixer = TimeEmbeddingMixer(c_time, c_out, act=act)
        self.conv_a = ConvBlock(c_in, c_out, act=act)
        self.conv_b = ConvBlock(c_out, c_out, act=act)
        if c_in != c_out:
            self.id_conv = nn.Conv2d(c_in, c_out, kernel_size=1)
        else:
            self.id_conv = None

        # Used for the Unet cross-link
        self.output = None

    def non_residual(self, x, t_emb):
        x = self.conv_a(x)
        x = self.time_mixer(x, t_emb)
        x = self.conv_b(x)
        return x

    def residual(self, x):
        if self.id_conv is not None:
            return self.id_conv(x)
        else:
            return x

    @jaxtyped(typechecker=beartype)
    def forward(
        self, x: InFeatureMapTensor, t_emb: TimeStepEmbeddingTensor
    ) -> OutFeatureMapTensor:
        x = self.non_residual(x, t_emb) + self.residual(x)
        self.output = x
        return x

In [30]:
xb = torch.randn(16, 3, 8, 8)

In [31]:
bs = xb.shape[0]
ts: TimeStepTensor = torch.linspace(-10, 10, bs)
tse = timestep_embedding(ts, 32)
tse.shape

torch.Size([16, 32])

In [32]:
ts

tensor([-10.0000,  -8.6667,  -7.3333,  -6.0000,  -4.6667,  -3.3333,  -2.0000,
         -0.6667,   0.6667,   2.0000,   3.3333,   4.6667,   6.0000,   7.3333,
          8.6667,  10.0000])

In [39]:
# |exports
class Downblock(nn.Module):
    """A superblock consisting of many downblocks of similar resolutions"""

    @beartype
    def __init__(
        self,
        c_time: int,
        c_in: int,
        c_out: int,
        downsample: bool = True,
        n_layers: int = 1,
        act: type[nn.Module] = nn.ReLU,
    ):
        super().__init__()
        self.c_time = c_time
        self.c_in = c_in
        self.c_out = c_out
        self.downsample = downsample
        self.n_layers = n_layers
        self.act = act

        self.convs = nn.ModuleList()
        self.convs.append(ResBlock(c_time, c_in, c_out, stride=1, act=act))
        for _ in range(n_layers - 1):
            self.convs.append(ResBlock(c_time, c_out, c_out, stride=1, act=act))
        self.downsampler = nn.Conv2d(c_out, c_out, kernel_size=3, stride=2, padding=1)

    @jaxtyped(typechecker=beartype)
    def forward(
        self, x: InFeatureMapTensor, t: TimeStepEmbeddingTensor
    ) -> OutFeatureMapTensor:
        for conv in self.convs:
            x = conv(x, t)
        if self.downsample:
            x = self.downsampler(x)
        return x

In [40]:
d = Downblock(32, 3, 2)
with torch.no_grad():
    yb = d(xb, tse)
xb.shape, yb.shape

(torch.Size([16, 3, 8, 8]), torch.Size([16, 2, 4, 4]))

In [41]:
# |exports
class Upblock(nn.Module):
    """A superblock consisting of many upblocks of similar resolutions
    and logic to use the activations of the counterpart downblock."""

    @beartype
    def __init__(
        self,
        c_time: int,
        c_in: int,
        c_out: int,
        upsample: bool = True,
        n_layers: int = 1,
        act: type[nn.Module] = nn.ReLU,
    ):
        super().__init__()
        self.c_time = c_time
        self.c_in = c_in
        self.c_out = c_out
        self.upsample = upsample
        self.n_layers = n_layers

        self.upsampler = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(c_in, c_in, kernel_size=3, padding=1),
        )
        self.convs = nn.ModuleList()
        for _ in range(n_layers - 1):
            self.convs.append(ResBlock(c_time, c_in * 2, c_in, stride=1))
        self.convs.append(ResBlock(c_time, c_in * 2, c_out, stride=1))

    @classmethod
    def from_downblock(cls, downblock):
        return cls(
            c_time=downblock.c_time,
            c_in=downblock.c_out,
            c_out=downblock.c_in,
            upsample=downblock.downsample,
            n_layers=downblock.n_layers,
            act=downblock.act,
        )

    @jaxtyped(typechecker=beartype)
    def forward(
        self, x: InFeatureMapTensor, downblock: Downblock, t: TimeStepEmbeddingTensor
    ) -> OutFeatureMapTensor:
        if self.upsample:
            x = self.upsampler(x)
        for up, down in zip(self.convs, reversed(downblock.convs)):
            x = up(torch.cat((x, down.output), dim=1), t)
        return x

In [42]:
u = Upblock.from_downblock(d)
with torch.no_grad():
    xp = u(yb, d, tse)
xb.shape == xp.shape

True

In [43]:
# |exports
class TimeEmbeddingMLP(nn.Module):
    """Small neural network to modify the "raw" time embeddings"""

    def __init__(self, c_in: int, c_out: int, act=nn.ReLU):
        super().__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.time_emb_mlp = nn.Sequential(
            nn.BatchNorm1d(c_in),
            nn.Linear(c_in, c_out),
            nn.ReLU(),
            nn.Linear(c_out, c_out),
        )

    @jaxtyped(typechecker=beartype)
    def forward(self, t: TimeStepTensor) -> TimeStepEmbeddingTensor:
        # Look up the sin/cos embedding  of the time step
        x = timestep_embedding(t, self.c_in).to(t.device)
        # Allow the model to slightly modify the embeddings
        x = self.time_emb_mlp(x)
        return x

In [44]:
# |exports
class Unet(nn.Module):
    """Diffusion U-net with a diffusion time dimension"""

    def __init__(
        self,
        nfs: Sequence[int],
        n_blocks: Sequence[int],
        color_channels: int = 3,
        act: type[nn.Module] = nn.ReLU,
    ):
        assert len(n_blocks) - 1 == len(nfs)
        super().__init__()

        self.time_embedding = TimeEmbeddingMLP(nfs[0], 4 * nfs[0])
        c_time = self.time_embedding.c_out

        # Since we use pre-activation ResBlocks, we need to use a Conv2d here
        # to avoid discarding pixel information
        self.start = nn.Conv2d(
            color_channels, nfs[0], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        )
        self.downblocks = nn.ModuleList()
        self.upblocks = nn.ModuleList()
        for c_in, c_out, n_layers in zip(nfs, nfs[1:], n_blocks):
            db = Downblock(
                c_time,
                c_in,
                c_out,
                n_layers=n_layers,
                act=act,
            )
            self.downblocks.append(db)
            self.upblocks.insert(0, Upblock.from_downblock(db))
        self.middle = ResBlock(c_time, nfs[-1], nfs[-1], stride=1, act=act)
        self.end = ConvBlock(nfs[0], color_channels, act=act)

    # Uniquely for a U-net module output dimensions must match the input dimensions
    @jaxtyped(typechecker=beartype)
    def forward(self, x_t: InFeatureMapTensor, t: TimeStepTensor) -> InFeatureMapTensor:
        te = self.time_embedding(t)
        _, c, _, _ = x_t.shape
        if c != self.start.in_channels:
            raise ValueError("model color channels must match input data channels")
        x = self.start(x_t)
        for db in self.downblocks:
            x = db(x, te)
        x = self.middle(x, te)
        for ub, db in zip(self.upblocks, reversed(self.downblocks)):
            x = ub(x, db, te)
        return self.end(x)

Let's make sure we can do a forward prop.

In [45]:
unet = Unet(
    nfs=(224, 448, 672, 896),
    n_blocks=(3, 2, 2, 1, 1),
    color_channels=3,
)

In [46]:
tb = torch.linspace(0, 1, 16)
xb.shape, tb.shape

(torch.Size([16, 3, 8, 8]), torch.Size([16]))

In [47]:
with torch.no_grad():
    yb = unet(xb, tb)

assert xb.shape == yb.shape

We'll use this size to quickly test the pipeline in later notebooks.

In [48]:
# |exports
def get_tiny_unet():
    return UnetLightning(
        nfs=(224, 448, 672, 896),
        n_blocks=(3, 2, 2, 1, 1),
        color_channels=1,
    )

In [22]:
#| hide
import nbdev

nbdev.nbdev_export()